In [1]:
import math
import pywt
import inspect
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import scipy.io.wavfile as wavfile
from scipy.interpolate import interp1d

# PRE-PROCESSING

In [2]:
input_rate, input_signal = wavfile.read('data/1clean_Selection.wav')
output_rate, output_signal = wavfile.read('data/1Selection.wav')

## Testing out pywt functions and reconstruction

In [3]:
# some settings:
wavetype = 'db10'
# wavelevel = 15

w = pywt.Wavelet(wavetype)
wavelevel = pywt.dwt_max_level(data_len=input_signal.shape[0], filter_len=w.dec_len)

# set a little lower:
# wavelevel = 1
print("Max wave level decomposition: " + str(wavelevel))

# these are floats, original is int16
input_coeffs1 = pywt.wavedec(input_signal[:,0].T, wavetype, level=wavelevel)
input_coeffs2 = pywt.wavedec(input_signal[:,1].T, wavetype, level=wavelevel)
# output_coeffs1 = pywt.wavedec(output_signal[:,0].T, wavetype, level=wavelevel)
# output_coeffs2 = pywt.wavedec(output_signal[:,1].T, wavetype, level=wavelevel)
print("input signal shape: " + str(input_signal.shape))

# reconstruction for left and right channel
recons1 = np.array([pywt.waverec(input_coeffs1, wavetype)]).astype('int16')
recons2 = np.array([pywt.waverec(input_coeffs2, wavetype)]).astype('int16')

# print(recons1.shape)

write_array = np.concatenate((recons1,recons2),axis=0).T
print('reconstruction shape: ' + str(write_array.shape))

# output wav for auditory test
wavfile.write('output/recons.wav', input_rate, write_array)

# get available wavelets
print(pywt.wavelist())

Max wave level decomposition: 16
input signal shape: (1810432, 2)
reconstruction shape: (1810432, 2)
['bior1.1', 'bior1.3', 'bior1.5', 'bior2.2', 'bior2.4', 'bior2.6', 'bior2.8', 'bior3.1', 'bior3.3', 'bior3.5', 'bior3.7', 'bior3.9', 'bior4.4', 'bior5.5', 'bior6.8', 'cgau1', 'cgau2', 'cgau3', 'cgau4', 'cgau5', 'cgau6', 'cgau7', 'cgau8', 'cmor', 'coif1', 'coif2', 'coif3', 'coif4', 'coif5', 'coif6', 'coif7', 'coif8', 'coif9', 'coif10', 'coif11', 'coif12', 'coif13', 'coif14', 'coif15', 'coif16', 'coif17', 'db1', 'db2', 'db3', 'db4', 'db5', 'db6', 'db7', 'db8', 'db9', 'db10', 'db11', 'db12', 'db13', 'db14', 'db15', 'db16', 'db17', 'db18', 'db19', 'db20', 'db21', 'db22', 'db23', 'db24', 'db25', 'db26', 'db27', 'db28', 'db29', 'db30', 'db31', 'db32', 'db33', 'db34', 'db35', 'db36', 'db37', 'db38', 'dmey', 'fbsp', 'gaus1', 'gaus2', 'gaus3', 'gaus4', 'gaus5', 'gaus6', 'gaus7', 'gaus8', 'haar', 'mexh', 'morl', 'rbio1.1', 'rbio1.3', 'rbio1.5', 'rbio2.2', 'rbio2.4', 'rbio2.6', 'rbio2.8', 'rbio3.1

In [4]:
print(input_signal[:10,0].T)
print(recons1[0,:10])

[ 0  0  0 -1 -2 -2 -3 -2 -1 -1]
[ 0  0  0 -1 -2 -2 -3 -2 -1 -1]


## prepare data for tf

In [5]:
# cut it into chunks

In [6]:
def wavelet_to_vector(input_raw, output_raw, chunk_size, wavelet_level, wavelet_type):
    current_set = 'input'
    
    w = pywt.Wavelet(wavelet_type)
    max_level = pywt.dwt_max_level(data_len=chunk_size, filter_len=w.dec_len)
    if wavelet_level > max_level:
        print('wavelet level too high. set to max level: ' + str(max_level))
        wavelet_level = max_level
    
    # short hacky loop
    while True:
        # select the correct set
        if current_set == 'input':
            data = input_raw
            amount_of_chunks = int(math.floor(data.shape[0]/chunk_size))
            input_list = []
        else:
            data = output_raw
            amount_of_chunks = int(math.floor(data.shape[0]/chunk_size))
            output_list = []

        
        index_range = (np.arange(amount_of_chunks) * chunk_size)
        indp = chunk_size - 1
        # for all chunks do this:
        for ind in index_range:
#             print(ind)
            sample1 = data[ind:ind+indp,0].T
            sample2 = data[ind:ind+indp,0].T
            coeffs1 = pywt.wavedec(sample1, wavelet_type, level=wavelet_level)
            coeffs2 = pywt.wavedec(sample2, wavelet_type, level=wavelet_level)
            unfolded1 = np.array([item for sublist in coeffs1 for item in sublist])
            unfolded2 = np.array([item for sublist in coeffs1 for item in sublist])
            vector = np.concatenate((unfolded1,unfolded2),axis=0)
            
            if current_set == 'input':
                input_list.append(vector)
            else:
                output_list.append(vector)
#             unf_arr = np.array(unfolded1)
#             print(unf_arr.shape)
#             print(len(unfolded))
            
            # for all coeff levels:
#             for i in range(len(coeffs1)):
#                 print(i)
            
        
        
        if current_set == 'output':
            break
        current_set = 'output'

    # convert lists to arrays
    input_arr = np.array(input_list)
    output_arr = np.array(output_list)
    
    # return level sizes for reconstruction
    level_sizes = []
    for cf in coeffs1:
        level_sizes.append(cf.shape[0])
    return [input_arr, output_arr, level_sizes]
# input_coeffs1[16].shape

# rows=samples, cols=dim
[input_matrix, output_matrix, level_sizes] = wavelet_to_vector(input_signal, output_signal, 1024, 6, 'db4')


In [7]:
a = np.array([4,5,7,4])
b = np.array([6,2,0,9])
# np.array([a, b])
# np.concatenate((a.T,b.T),axis=1)
# np.zeros([5,2])
# for cf in coef_test:
#     print(cf.shape)
# print(level_sizes)
np.array([a,b])
c = []
c.append(a)
c.append(b)
# np.array(c)
# a + b
# print(a.shape)
# np.concatenate((a,b),axis=0)
input_matrix.shape

(1768, 2124)

## set up tf

In [12]:
# Parameters
learning_rate = 0.001
training_iters = 10000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 10 # input_matrix.shape[1]
n_steps = input_matrix.shape[0]
# n_steps = 1
vec_size = input_matrix.shape[0]
n_hidden = 128 # hidden layer num of features
# n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
# batch size (None), seq length, dim
x = tf.placeholder("float", [None, n_steps, vec_size])
y = tf.placeholder("float", [None, n_steps, vec_size])

# Define weights
weights = {
    # Hidden layer weights => 2*n_hidden because of foward + backward cells
    'out': tf.Variable(tf.random_normal([n_hidden, n_input]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_input]))
}

In [19]:
def lstm_cell():
    # With the latest TensorFlow source code (as of Mar 27, 2017),
    # the BasicLSTMCell will need a reuse parameter which is unfortunately not
    # defined in TensorFlow 1.0. To maintain backwards compatibility, we add
    # an argument check here:
    if 'reuse' in inspect.getargspec(tf.contrib.rnn.BasicLSTMCell.__init__).args:
        return tf.contrib.rnn.core_rnn_cell.BasicLSTMCell(    # tf.contrib.rnn.core_rnn_cell.BasicLSTMCell(rnn_unit)
            n_hidden, forget_bias=0.0, state_is_tuple=True,
            reuse=tf.get_variable_scope().reuse)
    else:
        return tf.contrib.rnn.BasicLSTMCell(
            size, forget_bias=0.0, state_is_tuple=True)

def RNN(x, weights, biases):

    # reshape to [1, n_input]
#     x = tf.reshape(x, [-1, n_input])
    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, n_steps, 1)

    # Generate a n_input-element sequence of inputs
    # (eg. [had] [a] [general] -> [20] [6] [33])
#     x = tf.split(x,n_input,1)

    # 1-layer LSTM with n_hidden units.
#     rnn_cell = rnn.BasicLSTMCell(n_hidden,reuse=tf.get_variable_scope().reuse)

#     lstm_cell = lstm_cell()

    # generate prediction
    outputs, states = rnn.static_rnn(tf.contrib.rnn.core_rnn_cell.BasicLSTMCell(n_hidden, forget_bias=0.0, state_is_tuple=True, reuse=True), x, dtype=tf.float32)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [20]:
pred = RNN(x, weights, biases)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

ValueError: Variable rnn/basic_lstm_cell/weights/RMSProp/ already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-10-f07881d5b3bd>", line 5, in <module>
    optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)
  File "/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py", line 2821, in run_ast_nodes
    if self.run_code(code, result):


In [11]:
g = tf.Graph()
with g.as_default():
#     tf.reset_default_graph()
    def MYLSTM(x, weights, biases):

        # Prepare data shape to match `bidirectional_rnn` function requirements
        # Current data input shape: (batch_size, n_steps, n_input)
        # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)

        # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
        x = tf.unstack(x, n_steps, 1)

        # Define lstm cells with tensorflow
        # Forward direction cell
    #     with tf.variable_scope('forward'):
        lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
        # Backward direction cell
    #     with tf.variable_scope('backward'):
    #     lstm_bw_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)


        outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

        # Get lstm cell output
    #     try:
    #         outputs, _, _ = rnn(lstm_fw_cell, lstm_bw_cell, x,
    #                                               dtype=tf.float32)
    #     except Exception: # Old TensorFlow version only returns outputs not states
    #         outputs = tf.contrib.rnn(lstm_fw_cell, lstm_bw_cell, x,
    #                                         dtype=tf.float32)

        # Linear activation, using rnn inner loop last output
        return tf.matmul(outputs[-1], weights['out']) + biases['out']

    pred = MYLSTM(x, weights, biases)

    # Define loss and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

    # Evaluate model
    correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    # Initializing the variables
    init = tf.global_variables_initializer()

ValueError: Tensor("rnn/basic_lstm_cell/weights:0", shape=(1896, 512), dtype=float32_ref) must be from the same graph as Tensor("concat_2:0", shape=(?, 1896), dtype=float32).

In [ ]:
with tf.Session(graph=g) as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
#         batch_x, batch_y = mnist.train.next_batch(batch_size)
        ind = (step - 1) * batch_size
        batch_x = input_matrix[ind:ind+batch_size,:]
        batch_y = output_matrix[ind:ind+batch_size,:]
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc)
        step += 1
    print "Optimization Finished!"

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print "Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label})